In [1]:
import xarray as xr
import pandas as pd
import geoviews as gv
import geoviews.feature as gf

from cartopy import crs as ccrs

import geoviews.tile_sources as gts
from holoviews.operation.datashader import regrid

import numpy as np
gv.extension('bokeh')



ModuleNotFoundError: No module named 'xarray'

In [2]:
ds = xr.open_dataset(r'C:\Users\mrayson\cloudstor\Shared\InternalTideAtlas\NWS_internal_tide_atlas.nc')
ds

<xarray.Dataset>
Dimensions:    (depth: 100, frq: 5, latitude: 1000, longitude: 1750, modes: 4, time: 12)
Coordinates:
  * time       (time) datetime64[ns] 2013-07-01 2013-08-01 2013-09-01 ...
  * frq        (frq) float64 0.0001405 0.0001454 0.0001379 7.292e-05 6.76e-05
  * modes      (modes) float64 0.0 1.0 2.0 3.0
  * latitude   (latitude) float64 -25.0 -24.98 -24.96 -24.94 -24.92 -24.9 ...
  * longitude  (longitude) float64 107.5 107.5 107.5 107.6 107.6 107.6 107.6 ...
  * depth      (depth) float64 3.734 11.46 19.44 27.7 36.23 45.06 54.19 ...
Data variables:
    amp_b      (time, frq, modes, latitude, longitude) float64 ...
    phs_b      (time, frq, modes, latitude, longitude) float64 ...
    z          (latitude, longitude) float64 ...
    rho        (time, depth, latitude, longitude) float64 ...
Attributes:
    Name:          North West Shelf internal tide amplitude climatology
    Date created:  2018:11:05 21:59:57
    Author:        Matt Rayson
    Institution:   University of

In [3]:
esriocean = 'https://services.arcgisonline.com/ArcGIS/rest/services/Ocean/World_Ocean_Base/MapServer/tile/{Z}/{Y}/{X}'
esrinatgeo = 'http://services.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer/tile/{Z}/{Y}/{X}'
esrioceanref = 'http://services.arcgisonline.com/arcgis/rest/services/Ocean/World_Ocean_Reference/MapServer/tile/{Z}/{Y}/{X}'

In [4]:
dataset = gv.Dataset(ds, vdims='amp_b').redim.range(amp_b=(0,50.))


In [5]:
#%%opts Image [projection=crs.Geostationary()] (cmap='Greens') Overlay [xaxis=None yaxis=None]
image = dataset.to(gv.Image, ['longitude', 'latitude'], dynamic=True)\
    .options(width=700, height=570, alpha=0.8, cmap='afmhot_r', global_extent=False)

In [6]:
regrid(image)*\
gv.WMTS(esriocean).options(width=900, height=670, global_extent=False)*\
gv.WMTS(esrioceanref).options(width=900, height=670, alpha=0.9) 
    
# gv.feature.coastline

:DynamicMap   [time,frq,modes,depth]
   :Overlay
      .Image.I :Image   [longitude,latitude]   (amp_b)
      .WMTS.I  :WMTS   [Longitude,Latitude]
      .WMTS.II :WMTS   [Longitude,Latitude]

In [7]:
dataset_rho = gv.Dataset(ds, vdims='rho').redim.range(rho=(20,26.))
imagerho = dataset_rho.to(gv.Image, ['longitude', 'latitude'], dynamic=True)\
    .options(width=700, height=570, alpha=0.8, cmap='Spectral_r', global_extent=False)

regrid(imagerho)*\
gv.WMTS(esriocean).options(width=900, height=670, global_extent=False)*\
gv.WMTS(esrioceanref).options(width=900, height=670, alpha=0.9) 

:DynamicMap   [time,frq,modes,depth]
   :Overlay
      .Image.I :Image   [longitude,latitude]   (rho)
      .WMTS.I  :WMTS   [Longitude,Latitude]
      .WMTS.II :WMTS   [Longitude,Latitude]